In [34]:
import os
import random
import math
import time
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities


In [35]:
class gate_dlinkedList:
    def __init__(self, gate_id, bucket_side):
        self.gate_id = gate_id
        self.bucket_side = bucket_side
        self.lock = False
        self.gain = None
        self.next = None
        self.prev = None

    def set_next(self, next_gate):
        self.next = next_gate
        if next_gate:
            next_gate.prev = self
        return self.next
    
    def move_gate(self, new_gain):
        self.remove_gate()
        if new_gain in self.bucket_side.keys():
            self.bucket_side[new_gain].set_next(self)
        elif new_gain not in self.bucket_side.keys():
            self.bucket_side[new_gain] = self
        self.gain = new_gain
        
    def remove_gate(self):
        if self.gain in self.bucket_side.keys() and not self.next:
            if self.prev:
                self.bucket_side[self.gain] = self.prev
            else: #not self.prev
                del self.bucket_side[self.gain]
        if self.prev:
            self.prev.next = self.next
        if self.next:
            self.next.prev = self.prev
        self.next = None
        self.prev = None
        self.gain = None

    def __repr__(self):
        if self.next:
            next_id = self.next.gate_id  
        else:
            next_id = 'None'
        prev_id = self.prev.gate_id if self.prev else 'None'
        return f"Gate(Id:{self.gate_id},next:{next_id},prev:{prev_id},lock:{self.lock})"

In [36]:
class readFile:
    def __init__(self):
        self.circuit_data = None
        
    def read_circuit(self, circuit_folder_path):
        netD_file = None
        are_file = None
        for file in os.listdir(circuit_folder_path):
            if file.endswith('.netD'):
                netD_file = os.path.join(circuit_folder_path, file)
            if file.endswith('.are'):
                are_file = os.path.join(circuit_folder_path, file)
        if netD_file and are_file:
            self.circuit_data = self.parse_net_files(netD_file, are_file)
            return self.circuit_data
        else:
            print("Error Message: Missing Files")
            return None

    def parse_net_files(self, netD_file, are_file):
        # network information
        nets_out, nets_in, connections, nodes, modules, pad_offset = self.read_netD_file(netD_file)
        # module areas
        areas = self.read_are_file(are_file)
        # network information with pin directions
        self.circuit_data = {
            'areas': areas,
            'netD_out': nets_out,
            'netD_in': nets_in,
            'connections': int(connections),
            'nodes': int(nodes),
            'modules': int(modules),
            'pad_offset': int(pad_offset)
        }
        return self.circuit_data
    
    def read_are_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        module_areas = {}
        for line in lines:
            module_id, area = line.split()
            module_areas[module_id] = int(area)
        return module_areas

    def read_netD_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()

        self.circuit_info = tuple(item.replace('\n', '') for item in lines[0:5])
        _, connections, nodes, modules, pad_offset = self.circuit_info
        lines = lines[5:]

        nets_out = {}
        nets_in = {}
        current_key = None


        for line in lines:
            tokens = line.split()

            #populate dictionaries
            if tokens[0] not in nets_out.keys():
                nets_out[tokens[0]] = []
            if tokens[0] not in nets_in.keys():
                nets_in[tokens[0]] = []

            if tokens[1] == 's':
                current_key = tokens[0]
            elif tokens[1] == 'l':
                #nets out
                nets_out[current_key].append(tokens[0])
                #nets in
                nets_in[tokens[0]].append(current_key)
            
        return nets_out, nets_in, connections, nodes, modules, pad_offset

In [41]:
class FM_algo:
    def __init__(self, circuit_folder_path):
        self.start_time = time.time()
        self.circuit_data = readFile().read_circuit(circuit_folder_path)
        self.circuit_connections_in = self.circuit_data['netD_in']
        print('input connections', self.circuit_connections_in)
        self.circuit_connections_out = self.circuit_data['netD_out']
        print('output connections', self.circuit_connections_out)
        self.bucket_gains1 = dict()
        self.bucket_gains2 = dict()
        self.bucket_count1, self.bucket_count2 = 0, 0
        self.vertex = dict()
        initial_cutsize, best_cutsize, improvement, iter_count = self.fd_algo()
        runtime = time.time() - self.start_time

        print(f'Initial Cut Size: {initial_cutsize}')
        print('Fiduccia-Mattheyses Algorithm Complete!')
        print(f'Runtime: {runtime} seconds')
        print(f'Best Cut Size: {best_cutsize}')
        print(f'Total number of iterations: {iter_count}')
        print(f'Improvement percentage: {improvement}%')
        print(f'Number of Gates in Bucket 1: {self.bucket_count1}')
        print(f'Number of Gates in Bucket 2: {self.bucket_count2}')
        print(self.vertex)
        
    def fd_algo(self):
        #partition circuit
        self.partition1, self.partition2 = self.partition_circuit()
        print('parition1 ',self.partition1)
        print('parition2 ',self.partition2)
        #initialize buckets
        self.bucket_gains1 = self.initialize_buckets(self.partition1, self.bucket_gains1)
        self.bucket_gains2 = self.initialize_buckets(self.partition2, self.bucket_gains2)
        cutsize = self.calculate_cutsize()
        initial_cutsize = cutsize

        ##TODO: Begin Fiduccia-Mattheyses Algorithm below
        max_iter_count = 5
        iter_count = 0
        best_cutsize = float('inf') # Initialize the best cut size to inf
        
        while(True):
            iter_count += 1
            print(f"Number of Iterations: {iter_count}")
            last_cutsize = cutsize
            last_bucket1 = self.bucket_gains1
            last_bucket2 = self.bucket_gains2
            cutsize = self.fm_pass()
            if (cutsize < best_cutsize) and (math.isfinite(cutsize)): #if improvements
                best_cutsize = cutsize           
            if (cutsize >= last_cutsize) or (iter_count >= max_iter_count) or (last_cutsize == 1): #if no improvement
                #roll back to initial best buckets
                self.last_bucket1 = last_bucket1
                self.last_bucket2 = last_bucket2
                break   
            self.reinitialize_buckets()
        if math.isfinite(best_cutsize):
            improvement = round((1-best_cutsize/initial_cutsize)*100, 4)            
        return initial_cutsize, best_cutsize, improvement, iter_count

    def fm_pass(self):
        best_cutsize = float('inf')
        best_bucket1 = self.bucket_gains1
        best_bucket2 = self.bucket_gains2
        locked_gates_count = 0
        while (not self.check_gate_locks(locked_gates_count)): #Stop if all gates are locked
            max_gain_node, _ = self.calculate_max_gain()
            #move to opposite side of bucket && remove gate from bucket & lock gate
            max_gain_node.remove_gate()
            max_gain_node.lock = True
            locked_gates_count += 1
            print(f'FM progress: {locked_gates_count} / {len(self.vertex)}')
            if max_gain_node.bucket_side == self.bucket_gains1:
                max_gain_node.bucket_side = self.bucket_gains2
                self.bucket_count1 -= 1
                self.bucket_count2 += 1
            elif max_gain_node.bucket_side == self.bucket_gains2:
                max_gain_node.bucket_side = self.bucket_gains1
                self.bucket_count1 += 1
                self.bucket_count2 -= 1

            self.update_local_out_gains(max_gain_node)
            self.update_local_in_gains(max_gain_node)

            for net in self.circuit_connections_out[max_gain_node.gate_id]:
                net_node = self.vertex[net]
                net_side = net_node.bucket_side
                #Re-calculate gain
                new_gain = self.calculate_gain(net)
                if net_side == self.bucket_gains1:
                    net_node.move_gate(new_gain)
                else: #if net_side == self.bucket_gains2:
                    net_node.move_gate(new_gain)
                net_node.gain = new_gain
                current_cutsize = self.calculate_cutsize()
                #Rollback to best observed cutsize
                if current_cutsize < best_cutsize and current_cutsize >= 1:
                    best_cutsize = current_cutsize
                    best_bucket1 = self.bucket_gains1
                    best_bucket2 = self.bucket_gains2
                    
            # #iterate through INPUT connections to update gain buckets
            for net in self.circuit_connections_in[max_gain_node.gate_id]:
                net_node = self.vertex[net]
                net_side = net_node.bucket_side
                #Re-calculate gain
                new_gain = self.calculate_gain(net)
                if net_side == self.bucket_gains1:
                    net_node.move_gate(new_gain)
                else: #if net_side == self.bucket_gains2:
                    net_node.move_gate(new_gain)
                net_node.gain = new_gain
            current_cutsize = self.calculate_cutsize()
            #Rollback to best observed cutsize
            if current_cutsize < best_cutsize and current_cutsize >= 1:
                best_cutsize = current_cutsize
                best_bucket1 = self.bucket_gains1
                best_bucket2 = self.bucket_gains2

        self.bucket_gains1 = best_bucket1
        self.bucket_gains2 = best_bucket2
        new_cutsize = best_cutsize
        return new_cutsize

    def update_local_out_gains(self, moved_gate):
        for net in self.circuit_connections_out[moved_gate.gate_id]:
            net_node = self.vertex[net]
            net_side = net_node.bucket_side
            # Calculate new gain for the net
            new_gain = self.calculate_gain(net)
            # Update the gain of the gate in the same bucket
            if net_side == self.bucket_gains1:
                net_node.move_gate(new_gain)
            else: #if net_side == self.bucket_gains2:
                net_node.move_gate(new_gain)
            net_node.gain = new_gain
        
    def update_local_in_gains(self, moved_gate):
        for net in self.circuit_connections_in[moved_gate.gate_id]:
            net_node = self.vertex[net]
            net_side = net_node.bucket_side
            # Calculate new gain for the net
            new_gain = self.calculate_gain(net)
            # Update the gain of the gate in the same bucket
            if net_side == self.bucket_gains1:
                net_node.move_gate(new_gain)
            else: #if net_side == self.bucket_gains2:
                net_node.move_gate(new_gain)
            net_node.gain = new_gain
    
    def reinitialize_buckets(self):
        for gate, node in self.vertex.items():
            #unlock gate
            node.lock = False
            gain = self.calculate_gain(gate)
            node.gain = gain

            if gain in node.bucket_side.keys():
                node.bucket_side[gain].set_next(node)    
            else:
                node.bucket_side[gain] = node

    def check_gate_locks(self, locked_gates_count):
        # return True if all gates are locked
        return locked_gates_count == len(self.vertex) 

    def calculate_cutsize(self):
        cut_size = 0
        for gate, connected_gates in self.circuit_connections_out.items():
            current_gate_node = self.vertex[gate]
            current_gate_side = current_gate_node.bucket_side
            for connected_gate in connected_gates:
                connected_gate_node = self.vertex[connected_gate]
                connected_gate_side = connected_gate_node.bucket_side
                if current_gate_side != connected_gate_side:
                    cut_size += 1
                    break 
        return cut_size

    def calculate_max_gain(self):
        if self.bucket_gains1: max_gain1 = max(self.bucket_gains1.keys())
        else: max_gain1 = float('-inf')

        if self.bucket_gains2: max_gain2 = max(self.bucket_gains2.keys()) 
        else: max_gain2 = float('-inf')

        if max_gain1 > max_gain2: 
            selected_bucket = self.bucket_gains1  
        elif max_gain1 < max_gain2:
            selected_bucket = self.bucket_gains2
        else: #if max_gain1 == max_gain2 select the bigger bucket
            if self.bucket_count1 > self.bucket_count2:
                selected_bucket = self.bucket_gains1
            else: #self.bucket_count1 > self.bucket_count2
                selected_bucket = self.bucket_gains2

        overall_max_gain = max(max_gain1, max_gain2)

        if overall_max_gain != float('-inf'): max_gain_node = selected_bucket[overall_max_gain]  
        else: max_gain_node = None

        return max_gain_node, overall_max_gain
    
    def partition_circuit(self):
        circuit_nodes = list(self.circuit_connections_out.keys())
        random.shuffle(circuit_nodes)
        partition = len(circuit_nodes) // 2

        partition1 = circuit_nodes[:partition]
        partition2 = circuit_nodes[partition:] 
        return partition1, partition2
    
    def calculate_initial_gain(self, gate):
        external_sum = 0
        internal_sum = 0

        if gate in self.partition1:
            current_partition = self.partition1
        else:
            current_partition = self.partition2

        #output connections
        for next_gate in self.circuit_connections_out[gate]:
            if next_gate in current_partition:
                internal_sum += 1
            else:
                external_sum += 1
        
        #input connnections
        for prev_gate in self.circuit_connections_in[gate]:
            if prev_gate in current_partition:
                internal_sum += 1
            else:
                external_sum += 1
        
        #calculate gain
        gain = external_sum - internal_sum

        return gain


    def calculate_gain(self, gate):
        external_sum = 0
        internal_sum = 0

        current_partition = self.vertex[gate].bucket_side

        #output connections
        for next_gate in self.circuit_connections_out[gate]:
            next_gate_side = self.vertex[next_gate].bucket_side
            if next_gate_side == current_partition:
                internal_sum += 1
            else:
                external_sum += 1
        
        #input connnections
        for prev_gate in self.circuit_connections_in[gate]:
            prev_gate_side = self.vertex[prev_gate].bucket_side
            if prev_gate_side == current_partition:
                internal_sum += 1
            else:
                external_sum += 1
        
        #calculate gain
        gain = external_sum - internal_sum

        return gain
        
    def initialize_buckets(self, partition, bucket):
        for gate in partition:
            current_gate = gate_dlinkedList(gate, bucket)
            self.vertex[gate] = current_gate


            gain = self.calculate_initial_gain(gate)
            current_gate.gain = gain
            
            if gain not in bucket.keys():
                bucket[gain] = current_gate #gate must be doubly linked list OOC
            else:
                bucket[gain] = bucket[gain].set_next(current_gate)

            if bucket == self.bucket_gains1:
                self.bucket_count1 += 1
            else:
                self.bucket_count2 += 1
        return bucket 

In [43]:
# test 5 iteration
def main():
    circuit_folder_path = "test_simple_2"
    fm = FM_algo(circuit_folder_path)
if __name__ == "__main__":
    main()

input connections {'a123': [], 'a234': ['a123'], 'p345': ['a123']}
output connections {'a123': ['a234', 'p345'], 'a234': [], 'p345': []}
parition1  ['a234']
parition2  ['p345', 'a123']
Number of Iterations: 1
FM progress: 1 / 3
FM progress: 2 / 3
FM progress: 3 / 3
Initial Cut Size: 1
Fiduccia-Mattheyses Algorithm Complete!
Runtime: 0.0018930435180664062 seconds
Best Cut Size: 1
Total number of iterations: 1
Improvement percentage: 0.0%
Number of Gates in Bucket 1: 0
Number of Gates in Bucket 2: 3
{'a234': Gate(Id:a234,next:None,prev:None,lock:True), 'p345': Gate(Id:p345,next:None,prev:None,lock:True), 'a123': Gate(Id:a123,next:None,prev:None,lock:True)}


In [39]:
def main():
    circuit_folder_path = "test_simple"
    fm = FM_algo(circuit_folder_path)
if __name__ == "__main__":
    main()

input connections {'p1': [], 'a0': ['p1'], 'a1': ['p1'], 'a2': ['a0', 'a1'], 'a3': ['a0', 'a1'], 'p2': ['a2'], 'p3': ['a3']}
output connections {'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3'], 'p2': [], 'p3': []}
parition1  ['p3', 'a3', 'a2']
parition2  ['p1', 'a1', 'a0', 'p2']
Number of Iterations: 1
FM progress: 1 / 7
FM progress: 2 / 7
FM progress: 3 / 7
FM progress: 4 / 7
FM progress: 5 / 7
FM progress: 6 / 7
FM progress: 7 / 7
Number of Iterations: 2
FM progress: 1 / 7
FM progress: 2 / 7
FM progress: 3 / 7
FM progress: 4 / 7
FM progress: 5 / 7
FM progress: 6 / 7
FM progress: 7 / 7
Initial Cut Size: 3
Fiduccia-Mattheyses Algorithm Complete!
Runtime: 0.0014486312866210938 seconds
Best Cut Size: 1
Total number of iterations: 2
Improvement percentage: 66.6667%
Number of Gates in Bucket 1: 1
Number of Gates in Bucket 2: 6
{'p3': Gate(Id:p3,next:None,prev:None,lock:True), 'a3': Gate(Id:a3,next:None,prev:None,lock:False), 'a2': Gate(Id:a2,next:None

In [44]:
def main():
    circuit_folder_path = "test1"
    fm = FM_algo(circuit_folder_path)
if __name__ == "__main__":
    main()

input connections {'p198': [], 'a8117': ['p198', 'a7337', 'a9851', 'a7337', 'a9205', 'a9205', 'a9205', 'a9205', 'a5346'], 'a3044': ['a10425', 'a10425', 'a10425', 'a10425', 'a5154', 'a5154', 'a11352', 'a11352', 'a11352', 'a11352', 'a10425'], 'p98': ['a3044'], 'a3767': ['a6413', 'a6413', 'a6413', 'a6413', 'a3728', 'a5346'], 'p245': ['a3767'], 'a4992': ['a3981', 'a3981', 'a10461', 'a10461', 'a10461', 'a10461', 'a10425'], 'p231': ['a4992'], 'a6837': ['a9205', 'a9205', 'a9205', 'a9205', 'a12327', 'a6871', 'a12327', 'a11654', 'a11654', 'a11654', 'a11654', 'a8118'], 'p9': ['a6837'], 'a9877': ['a3418'], 'p52': ['a9877'], 'p146': [], 'a2553': ['p146', 'a9415', 'a2559', 'a5731'], 'a3427': ['a11677', 'a3767', 'a11677', 'a3767', 'a11677', 'a3767', 'a11677', 'a3767', 'a8793', 'a878', 'a8793', 'a5346'], 'p18': ['a3427'], 'a7784': ['a10972', 'a10972', 'a10972', 'a10972', 'a8321', 'a3881'], 'p17': ['a7784'], 'a6569': ['a3186'], 'p101': ['a6569'], 'p22': [], 'a459': ['p22', 'a2553', 'a2553', 'a2553', '

KeyboardInterrupt: 